# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df=pd.read_csv('../WeatherPy/output_data/Cities_Data.csv')
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.59,26.89,18.73,85.0,1.0,5.83,ZA,1.602781e+09
1,Port Elizabeth,-33.92,25.57,17.00,93.0,40.0,6.20,ZA,1.602782e+09
2,Tasiilaq,65.61,-37.64,5.00,75.0,75.0,3.10,GL,1.602782e+09
3,Cockburn Town,21.46,-71.14,28.44,78.0,5.0,5.37,TC,1.602782e+09
4,Zemio,5.03,25.14,23.63,87.0,33.0,0.77,CF,1.602782e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

# assign locations and humidity from the city_df
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [4]:
# using coordinate of Hong Kong as the map centre with zoom level set
fig = gmaps.figure(center=(22.302711, 114.177216), zoom_level=1.6)

# giving the param for heatmap
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# condition is Max Temp: 21-25ºc, 0 Cloudiness & Wind Speed < 5m/s
narrowed_city_df = city_df.loc[(city_df["Max Temp"] > 18) & (city_df["Max Temp"] < 22) & (city_df["Cloudiness"] < 10) \
                               & (city_df["Wind Speed"] < 4)& (city_df["Wind Speed"] > 0.5), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df=narrowed_city_df.sort_values(by="City", ascending=True)
narrowed_city_df.reset_index(inplace=True)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# remove previous index column and assign to hotel_df
hotel_df = narrowed_city_df.drop(columns="index")
hotel_df

# add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = np.nan

# create a list to hold hotel names
hotel = []

for i in range(len(hotel_df)):
    # set parameters to search for hotels with 5000 meters.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": (f'{hotel_df.loc[i]["Lat"]},{hotel_df.loc[i]["Lng"]}'), 
        "rankby": "distance",
        "type": "lodging",
        "key": g_key,
    }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel.append(results[0]["name"])
    except:
        hotel.append("")

# assign hotel into dataframe 
hotel_df["Hotel Name"] = hotel
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Abha,18.22,42.51,21.47,10.0,0.0,2.60,SA,1.602782e+09,شقق مفروشة
1,Barranca,-10.75,-77.77,20.27,66.0,6.0,3.97,PE,1.602782e+09,Familia Ciprianos Bodega
2,Chuy,-33.70,-53.46,19.98,68.0,0.0,3.85,UY,1.602781e+09,Gaboto
3,Constitucion,-35.33,-72.42,18.27,62.0,0.0,3.85,CL,1.602782e+09,Hostal Picapiedra
4,Faya,18.39,42.45,21.50,10.0,0.0,2.60,SA,1.602782e+09,مقبرة ال مجمل
5,Izmit,40.77,29.92,19.00,82.0,7.0,1.50,TR,1.602782e+09,Annem vodafon
6,Kharan,28.58,65.42,18.22,21.0,0.0,3.01,PK,1.602782e+09,Ghani Hasrat House
7,Kruisfontein,-34.00,24.73,18.33,83.0,0.0,1.79,ZA,1.602782e+09,Oyster Bay House Rental
8,Mustafakemalpasa,40.04,28.41,21.54,68.0,0.0,3.10,TR,1.602782e+09,Sabuncu Apartman
9,Saint George,37.10,-113.58,21.80,25.0,1.0,1.73,US,1.602782e+09,Jubilee Home


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))